# Carga, Calidad y Limpieza de Datos

### Importar Datos

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [ ]:
import os
import csv

# — CONFIGURA AQUÍ: —
folder = 'C:/Users/User/Desktop/TESIS/Datasets'

# Lista *manual* de tus 4 CSV originales:
input_files = [
    'Casos_Fraude.csv',   # ➔ FRAUDE = 1
    'BasePaisesNoComunes.csv',  # ➔ FRAUDE = 0
    'BasePaisesMasoComunes.csv',  # ➔ FRAUDE = 0
    'BaseEcuadorBolivia.csv',  # ➔ FRAUDE = 0
]

# Indica qué archivo de la lista es de fraude
flag_map = {
    'Casos_Fraude.csv': 1
}

# Columnas esperadas y posición de CELDA_A_DES (índice Python 16)
EXPECTED_FIELDS     = 26
CELDA_A_DES_IDX     = 16
ENCODING_IN         = 'ISO-8859-1'

# Ruta del CSV final
output_csv = os.path.join(folder, 'consolidado_final.csv')

with open(output_csv, 'w', newline='', encoding='utf-8') as fout:
    writer = csv.writer(fout)
    header_written = False

    for fname in input_files:
        path = os.path.join(folder, fname)
        if not os.path.isfile(path):
            raise FileNotFoundError(f"No encontré el archivo {fname} en {folder}")

        fraude_flag = flag_map.get(fname, 0)
        with open(path, 'r', encoding=ENCODING_IN) as fin:
            reader = csv.reader(fin)

            for i, row in enumerate(reader):
                if i == 0:
                    # Primera fila: cabecera
                    if not header_written:
                        writer.writerow(row + ['FRAUDE'])
                        header_written = True
                    continue

                # Si la fila tiene MÁS de 26 campos, fusionamos todos los extras en CELDA_A_DES
                if len(row) > EXPECTED_FIELDS:
                    extras = len(row) - EXPECTED_FIELDS
                    merged = ','.join(row[CELDA_A_DES_IDX : CELDA_A_DES_IDX + extras + 1])
                    row = (
                        row[:CELDA_A_DES_IDX]
                        + [merged]
                        + row[CELDA_A_DES_IDX + extras + 1 :]
                    )

                # Añadimos el flag al final y escribimos
                writer.writerow(row + [str(fraude_flag)])

print(f'✔️   consolidado_final.csv generado con éxito en "{output_csv}"')




✔️   consolidado_final.csv generado con éxito en "C:/Users/User/Desktop/TESIS/Datasets\consolidado_final_nosupe.csv"


In [5]:
# 1. Ruta al CSV (usa raw string o dobles \\ en Windows)
path = r'C:\Users\User\Desktop\TESIS\Datasets\DataSetNoSupervisado.csv'

# 2. Leer con pandas
df = pd.read_csv(
    path,
    sep=',',           # separador
    encoding='utf-8',  # coincide con el encoding de salida
    dtype=str,         # carga todas las columnas como texto (object)
    engine='python'    # más tolerante con quoting
)

# 3. Verifica
print(df.shape)  # (filas, 27 columnas esperadas)
print(df.columns.tolist())
df.head()

(1581115, 26)
['TIM_NUMBER', 'OTHER_NUMBER', 'NUMERO_A', 'NUMERO_B', 'NUMERO_MARCADO', 'DURACION_SEGUNDOS', 'DURACION_MINUTOS', 'MES', 'DIA', 'FECHAINICIO', 'FECHAFIN', 'PNI', 'TIPO_TRAFICO', 'SENTIDO_TRAFICO', 'OPER_A_DES', 'OPER_B_DES', 'CELDA_A_DES', 'CELDA_B_DES', 'LAC_A_LOCALIDAD', 'LAC_B_LOCALIDAD', 'IMEI_A', 'IMSI_A', 'IMEI_B', 'IMSI_B', 'CODIGODEPAIS', 'PAIS']


,TIM_NUMBER,OTHER_NUMBER,NUMERO_A,NUMERO_B,NUMERO_MARCADO,DURACION_SEGUNDOS,DURACION_MINUTOS,MES,DIA,FECHAINICIO,...,CELDA_A_DES,CELDA_B_DES,LAC_A_LOCALIDAD,LAC_B_LOCALIDAD,IMEI_A,IMSI_A,IMEI_B,IMSI_B,CODIGODEPAIS,PAIS
0,51900082950,56947272480,51900082950,56947272480,NaN,828,13.8,202503,01/03/2025,01/03/2025 11:20:06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,CHILE
1,51900584897,18099145186,51900584897,18099145186,NaN,21,.35,202503,01/03/2025,01/03/2025 09:19:40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,ESTADOS UNIDOS
2,51900943631,59170602659,51900943631,59170602659,NaN,89,1.48,202503,01/03/2025,01/03/2025 15:56:53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,591,BOLIVIA
3,51901350233,56947272553,51901350233,56947272553,NaN,143,2.38,202503,01/03/2025,01/03/2025 16:15:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,CHILE
4,51902102545,584127570017,51902102545,584127570017,191200584127570017,1924,32.07,202503,01/03/2025,01/03/2025 14:29:02,...,LIU0753@CANTUARIAS 270@MIRAFLORES@LIMA@LIMA@-7...,NaN,NaN,NaN,351016094387260,716101678707142,NaN,NaN,58,VENEZUELA


### Formateo de Línea

In [6]:
import random

# 2. Filtra solo los que empiezan por '51'
df = df[df['NUMERO_A'].str.startswith('51', na=False)].copy()

# 3. Separa el prefijo '51' para trabajar solo con la parte móvil
df['rest'] = df['NUMERO_A'].str[2:]

# 4. Genera un mapeo único original → anónimo
#    – Tomamos los valores únicos de rest que empiezan con '9'
originales = [x for x in df['rest'].unique() if x.startswith('9')]

#    – Definimos longitud (en tu caso 9 dígitos) y rango de nuevos números que empiecen con '8'
n = len(originales[0])                # 9
start = 8 * 10**(n-1)                 # 800000000
end   = 9 * 10**(n-1)                 # 900000000 (exclusive)

#    – Extraemos tantos números aleatorios **sin repetición** como originales haya
anonimos = random.sample(range(start, end), k=len(originales))

#    – Construimos el diccionario mapping: {'996803922':'803531244', …}
mapping = {orig: str(anon).zfill(n) for orig, anon in zip(originales, anonimos)}

# 5. Aplica el mapping de forma determinista
df['rest_masked'] = df['rest'].map(mapping)

# 6. Reconstruye el teléfono anonimizando
df['NUMERO_A_MASKED'] = '51' + df['rest_masked']

# 7. (Opcional) Si quieres hacer lo mismo con numero_b:
#    – Repite el paso 3–6 cambiando 'numero_a' por 'numero_b' y otro mapping distinto.

# 8. Limpia columnas intermedias
df = df.drop(columns=['rest','rest_masked'])

# 9. Ya tienes tu df con sólo números que empezaban en 51 
#    y con una columna nueva ('numero_a_masked') donde:
#     • siguen empezando con 51  
#     • ahora llevan un ‘8’ inicial en lugar de ‘9’  
#     • cada número original siempre mapea al mismo anónimo  
#     • ningún anónimo se repite

print(df[['NUMERO_A','NUMERO_A_MASKED']].head())

      NUMERO_A NUMERO_A_MASKED
0  51900082950     51893914429
1  51900584897     51867560817
2  51900943631     51844642903
3  51901350233     51870126489
4  51902102545     51879368992


In [7]:



mapping_b = {}        # dict: original_rest → anon_rest
used_anons = set()    # para evitar repeticiones

masked = []
for num, pref in zip(df['NUMERO_B'], df['CODIGODEPAIS']):
    if isinstance(num, str) and num.startswith(pref):
        rest = num[len(pref):]            # extrae la parte variable
        if rest not in mapping_b:
            length = len(rest)
            # genera un número aleatorio de la misma longitud:
            # entre 0 y 10^length-1, luego rellena con ceros si hiciera falta
            while True:
                candidate = str(random.randint(0, 10**length - 1)).zfill(length)
                if candidate not in used_anons:
                    used_anons.add(candidate)
                    mapping_b[rest] = candidate
                    break
        masked.append(pref + mapping_b[rest])
    else:
        masked.append(None)

df['NUMERO_B_MASKED'] = masked

# Guardar resultado
df.to_csv('consolidado_final_con_numero_b_masked.csv', index=False, encoding='utf-8')

# Verifica longitudes
print("Original lengths:", df['NUMERO_B'].dropna().str.len().unique())
print("Masked lengths:",   df['NUMERO_B_MASKED'].dropna().str.len().unique())



Original lengths: [11 12 13 10 24 15  9 20 25 17 22 21 14 23 18 19 16  8]
Masked lengths: [11 12 13 24 15  9 10 20 25 17 22 21 14 23 18 19 16  8]


In [8]:
output_path2 = 'C:/Users/User/Desktop/TESIS/Datasets/EnmascaradoDataSetNoSupervisado.csv'

df.to_csv(output_path2, index=False)

In [1]:
df

NameError: name 'df' is not defined